# Partitions:

Partition basically is a logical chunk of a large distributed data set. It provides the possibility to distribute the work across the cluster, divide the task into smaller parts, and reduce memory requirements for each node. Partition is the main unit of parallelism in Apache Spark.

vs coalesce

- repartition redistributes the data evenly, but at the cost of a shuffle
- coalesce works much faster when you reduce the number of partitions because it sticks input partitions together
- coalesce doesn’t guarantee uniform data distribution
- coalesce is identical to a repartition when you increase the number of partitions

# Window Functions

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

simpleData = (("James", "Sales", 3000), \
              ("Michael", "Sales", 4600),  \
              ("Robert", "Sales", 4100),   \
              ("Maria", "Finance", 3000),  \
              ("James", "Sales", 3000),    \
              ("Scott", "Finance", 3300),  \
              ("Jen", "Finance", 3900),    \
              ("Jeff", "Marketing", 3000), \
              ("Kumar", "Marketing", 2000),\
              ("Saif", "Sales", 4100) \
             )

columns= ["employee_name", "department", "salary"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

##### Ranking Functions

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number,col,rank,dense_rank

windowSpec  = Window.partitionBy("department").orderBy(col("salary").desc())

df.withColumn("row_number",row_number().over(windowSpec)).show()

In [ ]:
df.withColumn("rank",rank().over(windowSpec)).show()

##### Analytic Functions

In [ ]:
from pyspark.sql.functions import cume_dist    
df.withColumn("cume_dist",cume_dist().over(windowSpec)).show()

In [ ]:
from pyspark.sql.functions import lag    
df.withColumn("lag",lag("salary",2).over(windowSpec)).show()

##### Aggregate Functions

In [ ]:
windowSpecAgg  = Window.partitionBy("department")
from pyspark.sql.functions import col,avg,sum,min,max,row_number 
df.withColumn("row",row_number().over(windowSpec)) \
  .withColumn("avg", avg(col("salary")).over(windowSpecAgg)) \
  .withColumn("sum", sum(col("salary")).over(windowSpecAgg)) \
  .withColumn("min", min(col("salary")).over(windowSpecAgg)) \
  .withColumn("max", max(col("salary")).over(windowSpecAgg)) \
  .where(col("row")==1).select("department","avg","sum","min","max") \
  .show()
